In [1]:
import pandas as pd
import time
import numpy as np
from datetime import date
from sqlalchemy import create_engine
from dateutil.relativedelta import relativedelta

In [252]:
# actual payments
df_fact = pd.read_excel(r'path\rs_07_1.xlsx', sheet_name='Sheet 1', usecols='A:AK')
df_fact = df_fact.dropna(subset=['Вознаграждение за период'])

In [255]:
# parameters for calculating dealers revenue by tarif plan, dealer and monitor month (set by marketers)
df = pd.DataFrame()
engine = create_engine('oracle+cx_oracle://login:password@ip:port/?service_name=dwh')
query = f"""
select *
from dealer_rev.df_calc_param
"""
with engine.connect() as dwh_con:
    df_temp = pd.read_sql(query, dwh_con)
df_calc_param_f = pd.concat([df, df_temp])
engine.dispose() 
df_calc_param_f['group_number'] = df_calc_param_f[['group_number']].apply(lambda x: x.str.strip())
df_calc_param_f['Invcode_name'] = df_calc_param_f[['invcode_name']].apply(lambda x: x.str.strip())

In [257]:
# certain dealers (set by marketers)
df = pd.DataFrame()
engine = create_engine('oracle+cx_oracle://login:password@ip:port/?service_name=dwh')
query = f"""
select *
from dealer_rev.sdi
"""
with engine.connect() as dwh_con:
    df_temp = pd.read_sql(query, dwh_con)
df_sdi_f = pd.concat([df, df_temp])
engine.dispose() 

In [259]:
# distribution of dealers by dealer groups (set by marketers)
df = pd.DataFrame()
engine = create_engine('oracle+cx_oracle://login:password@ip:port/?service_name=dwh')
query = f"""
select *
from dealer_rev.sdi_grp
"""
with engine.connect() as dwh_con:
    df_temp = pd.read_sql(query, dwh_con)
df_sdi_grp_f = pd.concat([df, df_temp])
engine.dispose() 

In [260]:
df_fact = df_fact.merge(df_sdi_grp_f, how='left', left_on=['SDI'], right_on=['SDI'])
df_fact = df_fact[(df_fact['Отчётный месяц']>=df_fact['stime']) & (df_fact['Отчётный месяц']<=df_fact['etime'])]
df_fact['group_number'] = df_fact[['group_number']].apply(lambda x: x.str.strip())
df_fact['Номенклатура'] = df_fact[['Номенклатура']].apply(lambda x: x.str.strip())
df_all = df_fact

In [297]:
# set prediction period 
start_period = date(2023, 8, 1) 
end_period = date(2023, 9, 1)
date_list = pd.date_range(start_period, end_period, freq='MS') #ГГГГ-ММ-ДД
date_list

DatetimeIndex(['2023-08-01', '2023-09-01'], dtype='datetime64[ns]', freq='MS')

In [298]:
# predicted number of sales by group (set by marketers)
g1_prog = [220, 230, 240, 250, 200]
g2_prog = [5669, 5450, 5100, 4700, 3500]
g3_prog = [66427, 63424, 59758, 51431, 45000]
g4_prog = [3753, 3565, 3121, 2985, 2300]

In [278]:
# the function of calculating the "potential from the initial balance" in the predicting month
def calc_pot_pb(act, wd, pb, ppb, exp, exp_total, pot_pb, pot, perc, rs, rs_total):
    if (act == 0 and wd == 0):
        if exp + exp_total >= ppb:
            return 0
        if (exp + exp_total > pb and exp + exp_total < ppb):
            if rs > 0:
                return pot_pb
            if rs == 0:
                return pot_pb + (pb - exp_total) * perc
            else: 
                return -9999
        if exp + exp_total <= pb:
            return pot_pb + exp * perc
    else:
        if exp + exp_total >= ppb:
            return 0
        if (exp + exp_total > pb and exp + exp_total < ppb):
            if pot > 0:
                return pot_pb
            if pot == 0:
                return (exp + exp_total - pb) * perc
            else: 
                return -9999
        if exp_total <= pb:
            return pot_pb + exp * perc

In [268]:
# dealers revenue calculation function in the predicting month
def calc_rs(act, wd, pb, ppb, exp, exp_total, pot_pb, pot, perc, rs):
    if (act == 0 and wd == 0):
        if exp + exp_total >= ppb:
            return exp * perc + pot_pb + pot
        if (exp + exp_total > pb and exp + exp_total < ppb):
            if rs > 0:
                return exp * perc
            if rs == 0:
                return (exp + exp_total - pb) * perc
            else: 
                return -9999
        if exp_total <= pb:
            return 0
    else:
        return 0

In [269]:
# the function of calculating the "potential" in the predicting month
def calc_pot(act, wd, pb, ppb, exp, exp_total, pot_pb, pot, perc, rs_total):
    if (act == 0 and wd == 0):
        return 0
    else:
        if exp + exp_total >= ppb:
            return pot_pb + exp * perc 
        if (exp + exp_total > pb and exp + exp_total < ppb):
            if pot > 0:
                return pot + exp * perc
            if pot == 0:
                return (exp + exp_total - pb) * perc
            else: 
                return -9999
        if exp_total <= pb:
            return 0

In [270]:
# reward calculation function for predicted users
def calc_rs_new(pb, ppb, exp, exp_total, pot_pb, pot, perc, rs_total):
    if exp_total >= ppb:
        return exp * perc + pot_pb + pot
    if (exp_total > pb and exp_total < ppb):
        return (exp_total - pb) * perc - rs_total
    if exp_total <= pb:
        return 0

In [271]:
# the function of calculating the "potential from the initial balance" for new users
def calc_pot_pb_new(pb, ppb, exp, exp_total, pot_pb, pot, perc, rs, rs_total):
    if exp_total >= ppb:
        return 0
    if (exp_total > pb and exp_total < ppb):
        return exp * perc - rs - pot
    if exp_total <= pb:
        return pot_pb + exp * perc

In [272]:
# function for adjusting payouts in total no more than the limit
def corr_rs(limit, rs , rs_total):
    if rs_total + rs >= limit:
        return limit - rs_total
    else:
        return rs        

In [ ]:
%%time
for step, i in enumerate(date_list):
    df_all_last = df_all[df_all['Отчётный месяц'] == (i - relativedelta(months=1)).strftime("%Y-%m-%d")]
    df3 = df_all_last.copy()
    ### Predicting old users expenses
    # "promoting" users of the last actual month for a month ahead
    df3['Отчётный месяц'] = df3['Отчётный месяц'].apply(lambda x: x + relativedelta(months=1))
    df3['Отчётный период'] = df3['Отчётный период'] + 1
    # for the users we have "promoted", the parameters for calculating the dealer's remuneration should be replaced with 
    # the predicted month. 
    df4 = df3.merge(df_calc_param_f, how='left', left_on=['group_number', 'Номенклатура', 'Отчётный период'], 
                                                right_on=['group_number', 'invcode_name', 'rep_period'])
    df5 = df4[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура', 'group_number', 'SDI', 'DLR_ID', 'Дата активации', 
            'Дата регистрации', 'Отчётный период', 'Период наблюдения', 'Отчётный месяц', 'Первоначальный баланс',
            'Порог первоначального баланса', 'Потенциал с ПБ', 'Потенциал', 'rs', 'Оплаченные начисления B2C',
            'Итог оплаченные начисления B2C',  'Вознаграждение за период', 'Итого вознаграждение', 
            'Лимит по вознаграждению']]
    # remove users from the actual month who have completed the observation period in the predicting month or have reached limit
    df5 = df5[(df5['Итого вознаграждение'] < df5['Лимит по вознаграждению']) & (df5['Отчётный период'] <= 
                                                                                df5['Период наблюдения'])]
    # we introduce artificial columns for conditions in functions
    df5['Act_none'] = df5['Дата активации'].isna()
    df5['wd_none'] = df5['Дата регистрации'].isna()
    #  calculation of the current value of the "potential from the initial balance" in the predicting month
    df5['Потенциал с ПБ'] = df5.apply(lambda row: calc_pot_pb(row['Act_none'], row['wd_none'], row['Первоначальный баланс'], 
                                                              row['Порог первоначального баланса'], 
                                                              row['Оплаченные начисления B2C'], 
                                                              row['Итог оплаченные начисления B2C'],  
                                                              row['Потенциал с ПБ'], row['Потенциал'], 
                                                              row['rs'], row['Вознаграждение за период'], 
                                                              row['Итого вознаграждение']), axis=1)
    #  calculation actual dealers revenue in the predicting month
    df5['Вознаграждение за период'] = df5.apply(lambda row: calc_rs(row['Act_none'], row['wd_none'], 
                                                                    row['Первоначальный баланс'], 
                                                                    row['Порог первоначального баланса'],
                                                                    row['Оплаченные начисления B2C'], 
                                                                    row['Итог оплаченные начисления B2C'], 
                                                                    row['Потенциал с ПБ'], row['Потенциал'], 
                                                                    row['rs'], row['Итого вознаграждение']), axis=1)
    #  calculating actual "potential" in the predicting month
    df5['Потенциал'] = df5.apply(lambda row: calc_pot(row['Act_none'], row['wd_none'], 
                                                      row['Первоначальный баланс'], 
                                                      row['Порог первоначального баланса'],
                                                      row['Оплаченные начисления B2C'], 
                                                      row['Итог оплаченные начисления B2C'],
                                                      row['Потенциал с ПБ'], 
                                                      row['Потенциал'], 
                                                      row['rs'], row['Итого вознаграждение']), axis=1)
    # recalculation of the user's total expenses for the new month (add to the old value "Total", the value of this month's 
    # expenses)
    df5['Итог оплаченные начисления B2C'] = df5['Оплаченные начисления B2C'] + df5['Итог оплаченные начисления B2C']
    # adjustment of total payments no more than the limit
    df5['Вознаграждение за период'] = df5.apply(lambda row: corr_rs(row['Лимит по вознаграждению'], 
                                                                    row['Вознаграждение за период'], 
                                                                    row['Итого вознаграждение']), axis=1)
    # adjustment of total payments
    df5['Итого вознаграждение'] = df5['Итого вознаграждение'] + df5['Вознаграждение за период']
    df6 = df5[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура', 'group_number', 'SDI','DLR_ID', 'Дата активации', 
               'Дата регистрации', 'Отчётный период','Период наблюдения', 'Отчётный месяц', 'Первоначальный баланс', 
               'Порог первоначального баланса', 'Лимит по вознаграждению', 'Потенциал с ПБ', 'Потенциал', 'rs',
               'Оплаченные начисления B2C', 'Итог оплаченные начисления B2C', 'Вознаграждение за период', 
               'Итого вознаграждение']]
    # divide the DF into those who have a reporting period of 2,3,4 and the rest. For those with 2,3,4 periods, we take 
    # into account the survival rate, for the rest it does not change anymore.
    df70 = df6[(df6['Отчётный период'] != 2) & (df6['Отчётный период'] != 3) & (df6['Отчётный период'] != 4)]
    df72 = df6[(df6['Отчётный период'] == 2)]
    df73 = df6[(df6['Отчётный период'] == 3)]
    df74 = df6[(df6['Отчётный период'] == 4)]
    # adjust the number of users based on pre-calculated survival values (survival relative to the previous period)
    df72_n = df72.sample(frac=0.677)
    df73_n = df73.sample(frac=0.837)
    df74_n = df74.sample(frac=0.89)
    df8 = pd.concat([df70, df72_n, df73_n, df74_n])
    ### Predicting new users expenses
    # based on the users who registered in the month preceding the forecast, we create users who will appear in the predicting 
    # month by adjusting for the sales coefficient and assigning unique IDs.
    df21 = df_all[(df_all['Дата активации'].dt.date >= (i - relativedelta(months=1))) & (df_all['Дата активации'].dt.date < i)]
    df2n = df21[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура', 'group_number', 'SDI', 'DLR_ID','Дата активации', 
                 'Дата регистрации', 'Отчётный период', 'Период наблюдения', 'Лимит по вознаграждению', 'Отчётный месяц', 
                 'Первоначальный баланс', 'Порог первоначального баланса','Оплаченные начисления B2C',
                 'Итог оплаченные начисления B2C', 'Вознаграждение за период', 
                 'Итого вознаграждение']]
    # remove users from the actual month who have completed the observation period in the prediction month or have reached 
    # the payout limit
    df2n1 = df2n[(df2n['Итого вознаграждение'] < df2n['Лимит по вознаграждению']) & 
                 (df2n['Отчётный период'] <= df2n['Период наблюдения'])]
    # divide the DF by the number of tariff groups
    df2n_g1 = df2n1[(df2n1['trpl_group'] == 1)]
    df2n_g2 = df2n1[(df2n1['trpl_group'] == 2)]
    df2n_g3 = df2n1[(df2n1['trpl_group'] == 3)]
    df2n_g4 = df2n1[(df2n1['trpl_group'] == 4)]
    # adjusting the number of users in the forecast groups to the predicting values
    df2n_g1p = df2n_g1.sample(n=g1_prog[step], replace=True)
    df2n_g2p = df2n_g2.sample(n=g2_prog[step], replace=True)
    df2n_g3p = df2n_g3.sample(n=g3_prog[step], replace=True)
    df2n_g4p = df2n_g4.sample(n=g4_prog[step], replace=True)
    df2n_p = pd.concat([df2n_g1p, df2n_g2p, df2n_g3p, df2n_g4p])
    # changing the id to unique
    df2n_p['DIM_SUBS_ID'] = df2n_p['DIM_SUBS_ID'] + 1000000000000
    # change the activation date and the date of entering personal data for the forecast month
    df2n_p['Дата активации'] = i
    df2n_p['Дата регистрации'] = i
    df2n_p['Отчётный месяц'] = i
    df2n_p = df2n_p.drop(columns=['group_number','Лимит по вознаграждению','Период наблюдения','Вознаграждение за период',
                                  'Итого вознаграждение'])
    # we divide the DF with calculation parameters into tariff groups and leave only the necessary 'group_number'.
    # in the third tariff group, some dealers are paid with bonus conditions (determined by marketers)
    df_calc_param_f_g1 = df_calc_param_f[(df_calc_param_f['trpl_grp_id'] == 1) 
                                        & (df_calc_param_f['group_number'] == '1 new')
                                        & (df_calc_param_f['rep_period'] <= 9)]
    df_calc_param_f_g2 = df_calc_param_f[(df_calc_param_f['trpl_grp_id'] == 2) 
                                        & (df_calc_param_f['group_number'] == '1 new')
                                        & (df_calc_param_f['rep_period'] <= 9)]
    df_calc_param_f_g4 = df_calc_param_f[(df_calc_param_f['trpl_grp_id'] == 4) 
                                        & (df_calc_param_f['group_number'] == '1 new')
                                        & (df_calc_param_f['rep_period'] <= 9)]
    df_calc_param_f_g3_b = df_calc_param_f[(df_calc_param_f['trpl_grp_id'] == 3) 
                                        & (df_calc_param_f['group_number'] == '2 new')
                                        & (df_calc_param_f['rep_period'] <= 9)]
    df_calc_param_f_g3_bb = df_calc_param_f[(df_calc_param_f['trpl_grp_id'] == 3) 
                                        & (df_calc_param_f['group_number'] == '1 new')
                                        & (df_calc_param_f['rep_period'] <= 9)]
    # add the necessary group of dealers and parameters for the forecast
    df2n_p1 = df2n_p[(df2n_p['trpl_group'] == 1)].merge(df_calc_param_f_g1, how='left', 
                                                        left_on=[ 'Номенклатура', 'Отчётный период'], 
                                                        right_on=[ 'invcode_name', 'rep_period'])
    df2n_p2 = df2n_p[(df2n_p['trpl_group'] == 2)].merge(df_calc_param_f_g2, how='left', 
                                                        left_on=[ 'Номенклатура', 'Отчётный период'], 
                                                        right_on=[ 'invcode_name', 'rep_period'])
    df2n_p4 = df2n_p[(df2n_p['trpl_group'] == 4)].merge(df_calc_param_f_g4, how='left', 
                                                        left_on=[ 'Номенклатура', 'Отчётный период'], 
                                                        right_on=[ 'invcode_name', 'rep_period'])
    # coefficient for group 3 - how much with a bonus, how much without
    K3 = int(round(df2n_p[(df2n_p['trpl_group'] == 3)].shape[0] * 0.56, 0))
    # we divide the third group into two: with a bonus and without
    df2n_p3b = df2n_p[(df2n_p['trpl_group'] == 3)].iloc[:K3]
    df2n_p3bb =df2n_p[(df2n_p['trpl_group'] == 3)].iloc[K3:]
    df2n_p3b = df2n_p3b.merge(df_calc_param_f_g3_b, how='left', left_on=[ 'Номенклатура', 'Отчётный период'], 
                                                                right_on=[ 'invcode_name', 'rep_period'])
    df2n_p3bb = df2n_p3bb.merge(df_calc_param_f_g3_bb, how='left', left_on=[ 'Номенклатура', 'Отчётный период'], 
                                                                right_on=[ 'invcode_name', 'rep_period'])
    df2n_p1 = df2n_p1.rename(columns={'upper_limit':'Лимит по вознаграждению'})
    df2n_p2 = df2n_p2.rename(columns={'upper_limit':'Лимит по вознаграждению'})
    df2n_p4 = df2n_p4.rename(columns={'upper_limit':'Лимит по вознаграждению'})
    df2n_p3b = df2n_p3b.rename(columns={'upper_limit':'Лимит по вознаграждению'})
    df2n_p3bb = df2n_p3bb.rename(columns={'upper_limit':'Лимит по вознаграждению'})
    df2n_p1 = df2n_p1[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура','group_number', 'SDI', 'DLR_ID', 'Дата активации',
           'Дата регистрации', 'Отчётный период', 'rep_period', 'monitor_period', 'Лимит по вознаграждению', 'Отчётный месяц',
            'Первоначальный баланс', 'Порог первоначального баланса',
           'Оплаченные начисления B2C', 'Итог оплаченные начисления B2C', 'rs']]
    df2n_p2 = df2n_p2[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура','group_number', 'SDI', 'DLR_ID', 'Дата активации',
           'Дата регистрации', 'Отчётный период', 'rep_period', 'monitor_period', 'Лимит по вознаграждению', 'Отчётный месяц',
            'Первоначальный баланс', 'Порог первоначального баланса',
           'Оплаченные начисления B2C', 'Итог оплаченные начисления B2C', 'rs']]
    df2n_p4 = df2n_p4[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура','group_number', 'SDI', 'DLR_ID', 'Дата активации',
           'Дата регистрации', 'Отчётный период', 'rep_period', 'monitor_period', 'Лимит по вознаграждению', 'Отчётный месяц',
            'Первоначальный баланс', 'Порог первоначального баланса',
           'Оплаченные начисления B2C', 'Итог оплаченные начисления B2C', 'rs']]
    df2n_p3b = df2n_p3b[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура','group_number', 'SDI', 'DLR_ID', 'Дата активации',
           'Дата регистрации', 'Отчётный период', 'rep_period', 'monitor_period', 'Лимит по вознаграждению', 'Отчётный месяц',
            'Первоначальный баланс', 'Порог первоначального баланса',
           'Оплаченные начисления B2C', 'Итог оплаченные начисления B2C', 'rs']]
    df2n_p3bb = df2n_p3bb[['DIM_SUBS_ID', 'trpl_group', 'Номенклатура','group_number', 'SDI', 'DLR_ID', 'Дата активации',
           'Дата регистрации', 'Отчётный период', 'rep_period', 'monitor_period', 'Лимит по вознаграждению', 'Отчётный месяц',
            'Первоначальный баланс', 'Порог первоначального баланса',
           'Оплаченные начисления B2C', 'Итог оплаченные начисления B2C', 'rs']]
    # it is assumed that all the predicted users will enter personal data, so there is no check for their entry and there is 
    # no need to calculate the "potential"
    # zeroing the values
    df2n_p1['Потенциал'] = 0
    df2n_p1['Потенциал с ПБ'] = 0
    df2n_p1['Итого вознаграждение'] = 0
    df2n_p2['Потенциал'] = 0
    df2n_p2['Потенциал с ПБ'] = 0
    df2n_p2['Итого вознаграждение'] = 0
    df2n_p4['Потенциал'] = 0
    df2n_p4['Потенциал с ПБ'] = 0
    df2n_p4['Итого вознаграждение'] = 0
    df2n_p3b['Потенциал'] = 0
    df2n_p3b['Потенциал с ПБ'] = 0
    df2n_p3b['Итого вознаграждение'] = 0
    df2n_p3bb['Потенциал'] = 0
    df2n_p3bb['Потенциал с ПБ'] = 0
    df2n_p3bb['Итого вознаграждение'] = 0
    #  calculation of new dealer remuneration
    df2n_p1['Вознаграждение за период'] = df2n_p1.apply(lambda row: calc_rs_new(row['Первоначальный баланс'], 
                                                                                row['Порог первоначального баланса'],
                                                                                row['Оплаченные начисления B2C'], 
                                                                                row['Итог оплаченные начисления B2C'],  
                                                                                row['Потенциал с ПБ'], row['Потенциал'], 
                                                                                row['rs'], row['Итого вознаграждение']), axis=1)
    df2n_p2['Вознаграждение за период'] = df2n_p2.apply(lambda row: calc_rs_new(row['Первоначальный баланс'], 
                                                                                row['Порог первоначального баланса'],
                                                                                row['Оплаченные начисления B2C'], 
                                                                                row['Итог оплаченные начисления B2C'],
                                                                                row['Потенциал с ПБ'], row['Потенциал'], 
                                                                                row['rs'], row['Итого вознаграждение']), axis=1)
    df2n_p4['Вознаграждение за период'] = df2n_p4.apply(lambda row: calc_rs_new(row['Первоначальный баланс'], 
                                                                                row['Порог первоначального баланса'],
                                                                                row['Оплаченные начисления B2C'], 
                                                                                row['Итог оплаченные начисления B2C'], 
                                                                                row['Потенциал с ПБ'], row['Потенциал'], 
                                                                                row['rs'], row['Итого вознаграждение']), axis=1)
    df2n_p3b['Вознаграждение за период'] = df2n_p3b.apply(lambda row: calc_rs_new(row['Первоначальный баланс'], 
                                                                                  row['Порог первоначального баланса'],
                                                                                  row['Оплаченные начисления B2C'], 
                                                                                  row['Итог оплаченные начисления B2C'], 
                                                                                  row['Потенциал с ПБ'], row['Потенциал'], 
                                                                                  row['rs'], row['Итого вознаграждение']), 
                                                                                  axis=1)
    df2n_p3bb['Вознаграждение за период'] = df2n_p3bb.apply(lambda row: calc_rs_new(row['Первоначальный баланс'], 
                                                                                    row['Порог первоначального баланса'],
                                                                                    row['Оплаченные начисления B2C'], 
                                                                                    row['Итог оплаченные начисления B2C'], 
                                                                                    row['Потенциал с ПБ'], row['Потенциал'], 
                                                                                    row['rs'], row['Итого вознаграждение']), 
                                                                                    axis=1)
    # calculation of the new potential from the initial balance
    df2n_p1['Потенциал с ПБ'] = df2n_p1.apply(lambda row: calc_pot_pb_new(row['Первоначальный баланс'], 
                                                                          row['Порог первоначального баланса'], 
                                                                          row['Оплаченные начисления B2C'], 
                                                                          row['Итог оплаченные начисления B2C'],
                                                                          row['Потенциал с ПБ'], row['Потенциал'], 
                                                                          row['rs'], row['Вознаграждение за период'], 
                                                                          row['Итого вознаграждение']), axis=1)
    df2n_p2['Потенциал с ПБ'] = df2n_p2.apply(lambda row: calc_pot_pb_new(row['Первоначальный баланс'], 
                                                                          row['Порог первоначального баланса'], 
                                                                          row['Оплаченные начисления B2C'], 
                                                                          row['Итог оплаченные начисления B2C'], 
                                                                          row['Потенциал с ПБ'], row['Потенциал'], 
                                                                          row['rs'], row['Вознаграждение за период'], 
                                                                          row['Итого вознаграждение']), axis=1)
    df2n_p4['Потенциал с ПБ'] = df2n_p4.apply(lambda row: calc_pot_pb_new(row['Первоначальный баланс'], 
                                                                          row['Порог первоначального баланса'], 
                                                                          row['Оплаченные начисления B2C'], 
                                                                          row['Итог оплаченные начисления B2C'],  
                                                                          row['Потенциал с ПБ'], row['Потенциал'], 
                                                                          row['rs'], row['Вознаграждение за период'], 
                                                                          row['Итого вознаграждение']), axis=1)
    df2n_p3b['Потенциал с ПБ'] = df2n_p3b.apply(lambda row: calc_pot_pb_new(row['Первоначальный баланс'], 
                                                                            row['Порог первоначального баланса'], 
                                                                            row['Оплаченные начисления B2C'], 
                                                                            row['Итог оплаченные начисления B2C'],  
                                                                            row['Потенциал с ПБ'], row['Потенциал'], 
                                                                            row['rs'], row['Вознаграждение за период'], 
                                                                            row['Итого вознаграждение']), axis=1)
    df2n_p3bb['Потенциал с ПБ'] = df2n_p3bb.apply(lambda row: calc_pot_pb_new(row['Первоначальный баланс'], 
                                                                              row['Порог первоначального баланса'], 
                                                                              row['Оплаченные начисления B2C'], 
                                                                              row['Итог оплаченные начисления B2C'],  
                                                                              row['Потенциал с ПБ'], row['Потенциал'], 
                                                                              row['rs'], row['Вознаграждение за период'], 
                                                                              row['Итого вознаграждение']), axis=1)
    df2n_p1 = df2n_p1.rename(columns={'monitor_period':'Период наблюдения'})
    df2n_p2 = df2n_p2.rename(columns={'monitor_period':'Период наблюдения'})
    df2n_p3b = df2n_p3b.rename(columns={'monitor_period':'Период наблюдения'})
    df2n_p3bb = df2n_p3bb.rename(columns={'monitor_period':'Период наблюдения'})
    df2n_p4 = df2n_p4.rename(columns={'rep_period':'Период наблюдения'})
    df2n_f = pd.concat([df2n_p1, df2n_p2, df2n_p3b, df2n_p3bb, df2n_p4])
    # total DF of existing users and forecasted
    df_all = pd.concat([df8, df2n_f])
    df_all = df_all.drop(columns=['rs','rep_period','monitor_period'])
    df_all.to_excel(rf'D:\files_Dad\_work\IC-Service\Tasks\1\прогноз_{step}.xlsx', sheet_name='1', index=False)
    df_all = df_all.drop(columns='rs')